In [3]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
import requests
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [6]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [7]:
infos = list(set(extract_text_items(data)))

In [8]:
template = """
Fait un résumé concis des informations suivantes : {informations}
"""

In [9]:
prompt = PromptTemplate.from_template(template)


In [10]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [11]:
infos

['Lundi, "PIERRICK" est accompagnée de vents très forts de secteur Ouest au large de la pointe bretonne.Ces vents engendrent de fortes vagues de Sud-Ouest qui atteignent le littoral nord de la façade atlantique en fin de journée de lundi et sont particulièrement marquées sur l\'ouest du Finistère.De plus, les niveaux d\'eau sont élevés avec des coefficients de marée en hausse (109 à la pleine mer de lundi 18h) et une surélévation du niveau de la mer due aux conditions atmosphériques (surcote).La conjonction de ces très fortes vagues et de ces niveaux d\'eau élevés peut occasionner des submersions marines par franchissement de paquets de mer sur les zones exposées du littoral du Finistère lundi en fin de journée, placé en vigilance "vagues-submersion" orange.Dans une moindre mesure, le littoral du Morbihan, de la Loire-Atlantique et de la Vendée est placé en vigilance jaune "vagues-submersion".Heure de pleine mer à Brest lundi : 17h50.',
 'Conjonction de grandes marées et de fortes vagu

In [12]:
answer = chain.invoke(infos)

In [13]:
print(answer)


Le lundi, la dépression "PIERRICK" apporte des vents très forts et des vagues de Sud-Ouest au large de la pointe bretonne, créant un risque de submersion côtière sur le littoral nord de la façade atlantique en fin de journée. Les niveaux d'eau sont également élevés en raison des coefficients de marée en hausse et d'une surélévation due aux conditions atmosphériques. Les départements du Finistère, du Morbihan, de la Loire-Atlantique et de la Vendée sont placés en vigilance "vagues-submersion" orange et jaune. La tempête tardive pour la saison n'est pas exceptionnelle et présente une incertitude sur son positionnement exact. Le mardi, la dépression se décale vers l'est et engendre des vents violents et un risque de vagues-submersion en Manche. Les heures de pleine mer sont à surveiller pour les départements du Finistère, des Côtes-d'Armor, de l'Ille-et-Vilaine, de la Manche, du Calvados, de la Se


In [14]:
answer = chain.invoke(", ".join(infos))

In [15]:
print(answer)


Lundi, la dépression "PIERRICK" engendre des vents violents et de fortes vagues sur la pointe bretonne, avec un risque de submersion côtière dans le Finistère et les départements voisins. Les coefficients de marée sont élevés et une surélévation du niveau de la mer est attendue. Mardi, la dépression se déplace vers l'est, avec des vents forts et un risque de submersion sur la Manche et la façade atlantique. Les heures de pleine mer sont à surveiller pour les départements concernés.
